In [ ]:
import glob
import pandas as pd
import os
from astropy.io import ascii
import time
import numpy as np

In [ ]:
targdirs = glob.glob('/data/Dropbox (GPI)/GPIDATA/'+'/*')
if not targdirs:
    raise Exception('%s has not contents')

ct = 1
ct2 = 0

allfinaldat = pd.DataFrame();

for targdir in targdirs:
    dname = targdir.split('/')[-1]
    if (ct-1)%20 == 0:
        print("Starting %s, which is directory %g of %g..." % (dname,ct,len(targdirs)))

    # let's only look at H spec data
    txtlist = glob.glob(targdir+'/autoreduced/*H_Spec/contrast-*.txt')

    # create a dictionary of values for each target, then append to master Pandas DF
    tmpdict = {}
    for txtpath in txtlist:
        txtname = txtpath.split('/')[-1]  # contrast-Syyyymmdd-H.txt
        txtname1 = txtname.split('.')[-2] # contrast-Syyymmdd-H
        autodir = txtpath.split('/')[-2]  # yyyymmdd_H_Spec
        ifsmode = autodir.split('_')[-1]  # Spec

        dat = ascii.read(txtpath)

        # observation parameters
        tmpdict['datestr'] = dat.meta['Date']
        tmpdict['band'] = dat.meta['Band']
        tmpdict['mode'] = ifsmode
        tmpdict['fov_rot'] = dat.meta['Field Rotation']
        tmpdict['object']  = dat.meta['Object']
        tmpdict['object_'] = targdir.split('/')[-1]
        tmpdict['n_imgs'] = len(dat.meta['InputFiles'].split(','))

        # contrasts
        for sep in [0.25, 0.4, 0.8, 1.1]:
            #print sep
            tmp = np.abs(dat['Seps'] - sep)
            sepidx = tmp==tmp.min()
            if sum(sepidx) == 1:
                tmpdict['C%i_flat'%(sep*100)] = dat['L-Type'][sepidx][0]
                tmpdict['C%i_CH4'%(sep*100)] = dat['T-Type'][sepidx][0]
            else:
                print('WTF? 0 or >1 seps found for '+str(sep))
                tmpdict['C%i_flat'%(sep*100)] = np.nan
                tmpdict['C%i_CH4'%(sep*100)] = np.nan

        # append to master dataframe
        df = pd.DataFrame(tmpdict, index=[ct2])
        if len(allfinaldat) != 0:
            allfinaldat = allfinaldat.append(df)
        else:
            allfinaldat = df
        ct2+=1

    ct+=1

savedir = os.environ["HOME"]+'/Results/datatables/'

todaydate = time.strftime('%Y%m%d')
print('saving files to '+savedir)
#allfinaldat.to_excel(savedir+"Final_HSpec_Contrasts_"+todaydate+".xls", index=False,float_format='%.3e')
allfinaldat.to_csv(savedir+"Final_HSpec_Contrasts_"+todaydate+".txt", index=False,float_format='%.3e')
print('done')